# Dropout

paper: https://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf

## 문제상황

deep learning은 non-linear hidden layer가 여러 개 있어 복잡한 관계를 학습할 수 있지만, 훈련 데이터가 제한적일 경우 과적합이 발생 <br>
이를 방지하기 아래와 같은 기법이 적용되어 왔음 <br>
- early stop
- weight penalties ex) L1, L2 regularization
- soft weight sharing

계산 제약이 없다면 최적의 방법은 parameter의 모든 설정을 평균하여 예측을 조합하는 것이지만, 이는 계산 비용이 많이 듦. <br>
model combination은 성능을 향상시킬 수 있으나, 많은 네트워크를 따로 훈련하는 것은 힘들고, 충분한 훈련 데이터가 없을 수 있음. <br>

## Contribution

드롭아웃 기법은 위의 두 문제를 해결하기 위한 방법. <br>
**과적합을 방지하고, 무수히 많은 신경망 아키텍처를 효율적으로 결합.** <br>
네트워크에서 유닛을 임의로 제거하는 것으로 구현
- 각 유닛을 일정 확률로 유지
- input은 1에 가깝게 설정하는 것이 최적
- 외에는 0.5로 설정하는 것이 최적

## Dropout이란?

<img src="https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-14/04-dropout_overview.png" width="600" height="300"/>

노드의 일부분을 학습에 참여하지 않도록 하는 방법 <br>
드랍되는 노드는 매 mini-batch마다 binomial distribution을 활용하여 랜덤으로 설정 <br>
&nbsp;&nbsp;&nbsp;&nbsp; 노드의 드랍 확률 p는 hyperparameter

## 학습과 추론 방식의 차이

<img src="https://kh-kim.github.io/nlp_with_deep_learning_blog/assets/images/1-14/04-at_inference.png" width="500" height="200"/>

학습 시에는 노드의 일부분이 학습에 참여하지 않음 <br>
&nbsp;&nbsp;&nbsp;&nbsp; forward, backward <br>
반대로 추론 시에는 전체 노드가 참여 <br>
이 과정에서 발생하는 문제는 아래와 같음 <br>
> 출력 노드가 5개인 linear layer가 존재할 때, 만약 p=0.2이 출력 노드의 dropout에 적용됐다고 가정 <br>
> 학습 때는 평균 4개의 노드가 남아 평균적으로 5개 중 4개의 노드에서 값을 전달받을 것 <br>
> 하지만 추론 시 5개에서 값을 전달받을 것이기에 5/4만큼 더 큰 값을 전달받을 것 <br>
> 따라서, 추론 시 1-p에 해당하는 0.8를 곱하여 이 값을 상쇄 <br>
> pytorch에서는 학습 시 $\frac{1}{1-p}$를 출력에 곱하여 상쇄 <br>

## 사용 방법

일반적인 사용법은 activation 이후에 dropout <br>
최종 출력에는 dropout을 적용하지 않음 <br>

<br>

> ```python
> class Model(nn.Module):
>     def __init__(self, input_dim, hidden_dim, output_dim, dropout_prob=0.5):
>         super().__init__()
>         self.input_dim = input_dim
>         self.hidden_dim = hidden_dim
>         self.output_dim = output_dim
> 
>         self.linear1 = nn.Linear(input_dim, hidden_dim)
>         self.relu1 = nn.ReLU()
>         self.dropout1 = nn.Dropout(dropout_prob)  # 첫 번째 드롭아웃
>         self.linear2 = nn.Linear(hidden_dim, hidden_dim)
>         self.relu2 = nn.ReLU()
>         self.dropout2 = nn.Dropout(dropout_prob)  # 두 번째 드롭아웃
>         self.output = nn.Linear(hidden_dim, output_dim)
>     
>     def forward(self, x):
>         x = self.linear1(x)
>         x = self.relu1(x)
>         x = self.dropout1(x)  # 첫 번째 드롭아웃 적용
>         x = self.linear2(x)
>         x = self.relu2(x)
>         x = self.dropout2(x)  # 두 번째 드롭아웃 적용
>         x = self.output(x)
> 
>         return x
> ```

In [ ]:
# day29의 모듈화 코드를 적절하게 수정하여 dropout을 적용 후 dropout을 하지 않았을 때와 결과 비교
# tensorboardlogger의 filename을 적절하게 수정하여 tensorboard에서 식별하기 쉽도록 수정
# model layer에 dropout 추가
# (optinal) config에서 dropout ratio를 설정할 수 있도록 수정